<a href="https://colab.research.google.com/github/JaroslavHolecek/Teaching/blob/master/JupyterNotebook/SQL/Create_alter_delete_zadani.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL databáze — vytvoření, úprava a mazání tabulky

SQL databáze slouží k uložení a následnému zpracování dat. Samotné databáze si můžeme zjednodušeně představit jako excelovské tabulky — s tím rozdílem, že nad nimi stojí **SŘBD** (Systém Řízení Báze Dat), který umí s uloženými daty pracovat (ukládat, vyhledávat, provádět výpočty, hlídat integritu apod.).

---

## Jazyk SQL

**SQL** (Structured Query Language) je dotazovací jazyk, pomocí kterého komunikujeme s databází. V SQL příkazech **nepopisujeme, jak** se má výsledek získat, ale pouze **co chceme** za výsledek — o zbytek se postará SŘBD.

SQL příkazy dělíme do několika kategorií:

| Kategorie | Zkratka | Příkazy | Účel |
|---|---|---|---|
| Data Definition Language | **DDL** | `CREATE`, `ALTER`, `DROP` | Definice struktury (tabulky, sloupce) |
| Data Manipulation Language | **DML** | `INSERT`, `UPDATE`, `DELETE` | Manipulace s daty |
| Data Query Language | **DQL** | `SELECT` | Dotazování / čtení dat |
| Data Control Language | **DCL** | `GRANT`, `REVOKE` | Správa oprávnění |

V tomto notebooku se zaměříme na **DDL** — vytváření, úpravu a mazání tabulek.

---

## Konvence pojmenování

Při práci s databází dodržujeme tyto konvence:
- Názvy tabulek a sloupců píšeme **malými písmeny**, **bez diakritiky**, s podtržítky (`snake_case`): `automobily`, `pocet_sedadel`, `datum_vyroby`.
- SQL klíčová slova píšeme **VELKÝMI PÍSMENY**: `CREATE TABLE`, `NOT NULL`, `PRIMARY KEY`.
- Názvy tabulek volíme v **množném čísle**: `automobily` (ne `automobil`), `uzivatele` (ne `uzivatel`). Tabulka obsahuje více záznamů — proto množné číslo.

> **Pozn.:** V praxi existují oba přístupy (jednotné i množné číslo). Důležité je být **konzistentní** v celém projektu. Množné číslo je běžnější — používají ho frameworky jako Django, Rails, Laravel i většina tutoriálů.

> **⚠️ Poznámka:** Knihovna `mysql-connector-python` nepodporuje datový typ `TEXT` v některých kontextech. Místo něj používáme `CHAR(n)` nebo `VARCHAR(n)`, kde `n` je maximální počet znaků. V podkladech je uveden `VARCHAR` pro přehlednost. Tato poznámka platí pro všechny následující podklady.

## Instalace balíčku

In [ ]:
# Instalace knihovny (stačí spustit jednou)
! pip install mysql-connector-python


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# Připojení k databázi

Než začneme s databází pracovat, musíme se k ní připojit. Přihlašovací údaje importujeme z modulu `pripojeni.py` (nikdy je nepíšeme přímo do kódu).

In [ ]:
import mysql.connector
from pripojeni import *  # importuje konstanty HOST, USER, PASSWORD, DATABASE

# Připojení k databázi
mydb = mysql.connector.connect(
    host=HOST,
    user=USER,
    password=PASSWORD,
    database=DATABASE
)

# Kurzor — objekt, přes který posíláme SQL příkazy a čteme výsledky
mycursor = mydb.cursor()

# CREATE TABLE — vytvoření tabulky

Tabulka se v databázi vytvoří pomocí příkazu `CREATE TABLE`.

## Syntaxe

```sql
CREATE TABLE nazev_tabulky (
    nazev_sloupce datovy_typ [omezeni],
    ...
);
```

Při vytváření musíme zadat:
1. **Název tabulky** — v `snake_case`, bez diakritiky, množné číslo.
2. **Názvy sloupců** (atributů) — v `snake_case`, bez diakritiky.
3. **Datový typ** každého sloupce (`INT`, `VARCHAR(n)`, `DATE`, …).
4. Volitelně **omezení** (`NOT NULL`, `UNIQUE`, `PRIMARY KEY`, …).

### Jednoduchý příklad

In [ ]:
# SQL dotaz v trojitých uvozovkách — umožňuje zápis na více řádků
mycursor.execute("""
    CREATE TABLE automobily (
        id INT,
        spz CHAR(7),
        pocet_sedadel INT,
        max_rychlost INT,
        nosnost INT,
        nutna_kvalifikace VARCHAR(50),
        datum_vyroby DATE
    )
""")

# commit() potvrdí provedení příkazu (jako stisknutí Enter)
mydb.commit()

> **Tip:** Pokud chcete tabulku vytvořit pouze v případě, že ještě neexistuje, použijte `IF NOT EXISTS`:
> ```sql
> CREATE TABLE IF NOT EXISTS automobily ( ... );
> ```

# DROP TABLE — smazání tabulky

Příkaz `DROP TABLE` slouží ke **smazání celé tabulky** včetně všech dat.

```sql
DROP TABLE nazev_tabulky;
```

> **⚠️ Pozor:**
> - Smazání je **nevratné** — všechna data budou ztracena.
> - Tabulku nelze smazat, pokud na ni odkazuje jiná tabulka cizím klíčem (`FOREIGN KEY`). To lze však vypnout příkazem `SET FOREIGN_KEY_CHECKS = 0;` a poté zase zapnout `SET FOREIGN_KEY_CHECKS = 1`. Toto provádějte na vlastní nebezpečí a jen pokud je tabulka prázdná.
> - Tabulka, kterou chceme smazat, musí existovat (jinak dostaneme chybu). Můžeme použít `DROP TABLE IF EXISTS nazev_tabulky;`

In [ ]:
# Smazání tabulky vytvořené v předchozím příkladu
mycursor.execute("""DROP TABLE IF EXISTS automobily""")
mydb.commit()

# Omezení (constraints) sloupců

Ke sloupcům můžeme při vytváření tabulky přidat různá **omezení**:

| Omezení | Popis |
|---|---|
| `PRIMARY KEY` | Vlastní (primární) klíč — jednoznačně identifikuje každý záznam. Může být v tabulce **pouze jeden**. |
| `FOREIGN KEY` | Cizí klíč — odkazuje na primární klíč jiné tabulky (vytváří relaci). |
| `NOT NULL` | Hodnota **nesmí být prázdná** (NULL). |
| `UNIQUE` | Hodnota se **nesmí v daném sloupci opakovat**. |
| `AUTO_INCREMENT` | SŘBD automaticky přiřadí hodnotu o jednu vyšší než dosud nejvyšší. |
| `DEFAULT hodnota` | Pokud hodnota není vyplněna, uloží se tato **výchozí hodnota**. |

> Podrobnější vysvětlení naleznete v samostatném notebooku [Vlastnosti atributu](https://github.com/JaroslavHolecek/Teaching/tree/master/JupyterNotebook/SQL).

### Příklad s omezením

In [ ]:
# Nejdříve smažeme tabulku, pokud existuje z předchozího spuštění
mycursor.execute("""DROP TABLE IF EXISTS automobily""")
mydb.commit()

mycursor.execute("""
    CREATE TABLE automobily (
        id INT PRIMARY KEY AUTO_INCREMENT,
        spz CHAR(7) NOT NULL UNIQUE,
        pocet_sedadel INT NOT NULL,
        max_rychlost INT,
        nosnost INT,
        nutna_kvalifikace VARCHAR(50) NOT NULL DEFAULT 'B',
        datum_vyroby DATE
    )
""")

mydb.commit()

# ALTER TABLE — úprava tabulky

Příkaz `ALTER TABLE` slouží k úpravě **již existující** tabulky. Můžeme:

| Operace | Syntaxe |
|---|---|
| Přidat sloupec | `ALTER TABLE tabulka ADD nazev_sloupce datovy_typ;` |
| Smazat sloupec | `ALTER TABLE tabulka DROP COLUMN nazev_sloupce;` |
| Změnit datový typ | `ALTER TABLE tabulka MODIFY nazev_sloupce novy_typ;` |
| Přejmenovat sloupec | `ALTER TABLE tabulka CHANGE stary_nazev novy_nazev datovy_typ;` |
| Přidat omezení | `ALTER TABLE tabulka ADD CONSTRAINT ... ;` |

### Příklad — přidání sloupce

In [ ]:
# Vytvoříme tabulku bez sloupce datum_vyroby
mycursor.execute("""DROP TABLE IF EXISTS automobily""")
mydb.commit()

mycursor.execute("""
    CREATE TABLE automobily (
        id INT PRIMARY KEY AUTO_INCREMENT,
        spz CHAR(7) NOT NULL UNIQUE,
        pocet_sedadel INT NOT NULL,
        max_rychlost INT,
        nosnost INT,
        nutna_kvalifikace VARCHAR(50) NOT NULL DEFAULT 'B'
    )
""")
mydb.commit()

# Přidáme sloupec datum_vyroby pomocí ALTER TABLE
mycursor.execute("""ALTER TABLE automobily ADD datum_vyroby DATE""")
mydb.commit()

print("✅ Sloupec 'datum_vyroby' úspěšně přidán.")

# Přehled datových typů

V databázích existuje mnoho datových typů. Správné zvolení typu zefektivňuje chod databáze (zabírá méně místa, rychlejší vyhledávání).

| Typ | Popis | Příklad |
|---|---|---|
| `INT` | Celé číslo | `42`, `-7` |
| `FLOAT` / `DOUBLE` | Desetinné číslo (přibližné) | `3.14` |
| `DECIMAL(m, d)` | Přesné desetinné číslo (_m_ číslic, _d_ za tečkou) | `DECIMAL(10, 2)` → `12345678.99` |
| `CHAR(n)` | Řetězec **pevné** délky _n_ znaků | `CHAR(7)` → `"2A5 01 "` |
| `VARCHAR(n)` | Řetězec **proměnlivé** délky (max _n_ znaků) | `VARCHAR(100)` → `"Ahoj"` |
| `TEXT` | Dlouhý text (neomezená délka) | článek, popis |
| `DATE` | Datum (`YYYY-MM-DD`) | `2025-03-15` |
| `DATETIME` | Datum a čas | `2025-03-15 14:30:00` |
| `BOOLEAN` | Pravda / nepravda | `TRUE`, `FALSE` |

Kompletní přehled datových typů najdete na [W3Schools — SQL Data Types](https://www.w3schools.com/sql/sql_datatypes.asp).

---

# Odpojení od databáze

Na konci práce vždy uzavřeme kurzor i spojení:

In [ ]:
# Uklidíme po sobě a odpojíme se
mycursor.execute("""DROP TABLE IF EXISTS automobily""")
mydb.commit()

mycursor.close()
mydb.close()

print("✅ Odpojení proběhlo úspěšně.")

---

# Cvičení

Zde bude následovat série úkolů, díky kterým si ověříte, zda jste látku pochopili.

> V každém cvičení se nejprve připojte k databázi pomocí konstant z modulu `pripojeni`. Na konci se vždy odpojte.

## Cvičení 1
* Připojte se k databázi a vytvořte tabulku `uzivatele` s atributy:
  * `id` — `INT`
  * `jmeno` — `CHAR(10)`
  * `prijmeni` — `CHAR(20)`
  * `email` — `CHAR(50)`
* Nepřidávejte k atributům žádné další vlastnosti (omezení).

<details>
<summary>🔎 Očekávaný výstup (klikni pro zobrazení)</summary>

```
Tabulka vytvořena správně.
```

</details>

In [ ]:
import mysql.connector
from pripojeni import *

mydb = mysql.connector.connect(
    host=HOST, user=USER, password=PASSWORD, database=DATABASE
)
mycursor = mydb.cursor()

# TODO: zde vytvořte tabulku


mydb.commit()

# --- tuto část neměnit! ---
mycursor.execute("""DESCRIBE uzivatele""")
table = str(mycursor.fetchall())
expected = "[('id', 'int(11)', 'YES', '', None, ''), ('jmeno', 'char(10)', 'YES', '', None, ''), ('prijmeni', 'char(20)', 'YES', '', None, ''), ('email', 'char(50)', 'YES', '', None, '')]"
if table == expected:
    print("Tabulka vytvořena správně.")
else:
    print("Tabulka není dle zadání.\nVaše tabulka:")
    print(table)
    print("Očekávaná tabulka:")
    print(expected)

mycursor.execute("""DROP TABLE uzivatele""")
mydb.commit()
mycursor.close()
mydb.close()

## Cvičení 2
* Připojte se k databázi a vytvořte tabulku `uzivatele` s atributy a omezením:
  * `id` — `INT`, `PRIMARY KEY`, `AUTO_INCREMENT`
  * `jmeno` — `CHAR(10)`, `NOT NULL`
  * `prijmeni` — `CHAR(20)`, `NOT NULL`
  * `email` — `CHAR(50)`, `NOT NULL`, `UNIQUE`

<details>
<summary>🔎 Očekávaný výstup (klikni pro zobrazení)</summary>

```
Tabulka vytvořena správně.
```

</details>

In [ ]:
import mysql.connector
from pripojeni import *

mydb = mysql.connector.connect(
    host=HOST, user=USER, password=PASSWORD, database=DATABASE
)
mycursor = mydb.cursor()

# TODO: zde přijde váš kód →


mydb.commit()

# --- tuto část neměnit! ---
mycursor.execute("""DESCRIBE uzivatele""")
table = str(mycursor.fetchall())
expected = "[('id', 'int(11)', 'NO', 'PRI', None, 'auto_increment'), ('jmeno', 'char(10)', 'NO', '', None, ''), ('prijmeni', 'char(20)', 'NO', '', None, ''), ('email', 'char(50)', 'NO', 'UNI', None, '')]"
if table == expected:
    print("Tabulka vytvořena správně.")
else:
    print("Tabulka není dle zadání.\nVaše tabulka:")
    print(table)
    print("Očekávaná tabulka:")
    print(expected)

mycursor.execute("""DROP TABLE uzivatele""")
mydb.commit()
mycursor.close()
mydb.close()

## Cvičení 3
* Dopište kód tak, aby se tabulka `uzivatele` rozšířila o atribut `datum_narozeni` (`DATE`).
* Pro rozšíření použijte příkaz `ALTER TABLE`.

<details>
<summary>🔎 Očekávaný výstup (klikni pro zobrazení)</summary>

```
Tabulka vytvořena správně.
```

</details>

In [ ]:
import mysql.connector
from pripojeni import *

mydb = mysql.connector.connect(
    host=HOST, user=USER, password=PASSWORD, database=DATABASE
)
mycursor = mydb.cursor()

# --- tento příkaz neměnit! ---
mycursor.execute("""
    CREATE TABLE uzivatele (
        id INT,
        jmeno CHAR(10),
        prijmeni CHAR(20),
        email CHAR(50)
    )
""")
mydb.commit()

# TODO: zde přidejte sloupec datum_narozeni pomocí ALTER TABLE →


mydb.commit()

# --- tuto část neměnit! ---
mycursor.execute("""DESCRIBE uzivatele""")
table = str(mycursor.fetchall())
expected = "[('id', 'int(11)', 'YES', '', None, ''), ('jmeno', 'char(10)', 'YES', '', None, ''), ('prijmeni', 'char(20)', 'YES', '', None, ''), ('email', 'char(50)', 'YES', '', None, ''), ('datum_narozeni', 'date', 'YES', '', None, '')]"
if table == expected:
    print("Tabulka vytvořena správně.")
else:
    print("Tabulka není dle zadání.\nVaše tabulka:")
    print(table)
    print("Očekávaná tabulka:")
    print(expected)

mycursor.execute("""DROP TABLE uzivatele""")
mydb.commit()
mycursor.close()
mydb.close()

## Cvičení 4
* Připojte se k databázi.
* Vytvořte tabulku `testy` s atributy:
  * `id` — `INT`, `AUTO_INCREMENT`, `PRIMARY KEY`
  * `nazev` — `CHAR(15)`, `NOT NULL`, `UNIQUE`
* Pomocí příkazu `ALTER TABLE` přidejte sloupec `datum` (`DATE`, `NOT NULL`).
* Zkontrolujte výsledek přes **phpMyAdmin** (`dbs.spskladno.cz/myadmin`).
* Celou tabulku následně smažte.

<details>
<summary>🔎 Očekávaný výstup (klikni pro zobrazení)</summary>

```
✅ Tabulka 'testy' vytvořena.
✅ Sloupec 'datum' přidán.
✅ Tabulka 'testy' smazána.
```

</details>

In [ ]:
import mysql.connector
from pripojeni import *

mydb = mysql.connector.connect(
    host=HOST, user=USER, password=PASSWORD, database=DATABASE
)
mycursor = mydb.cursor()

# TODO: Celý úkol je na vás — výsledek si zkontrolujte přes dbs.spskladno.cz/myadmin


<p>Výsledek v MyAdmin</p>
<img width="500" src="https://raw.githubusercontent.com/JaroslavHolecek/Teaching/master/JupyterNotebook/SQL/Images/JyputerNotebook_checkExercise_SQLcreate.png">

In [ ]:
# TODO: Zde tabulku po kontrole správnosti smažte
# Nezapomeňte se na konci odpojit (mycursor.close(), mydb.close())
